In [ ]:
# Install and load necessary R packages
library(dplyr)
# library(e1071)
library(randomForest)
library(ggplot2)
library(stringr)

In [ ]:
setwd("D:\\Kaggle\\Titanic")
train <- read.csv("train.csv", sep =",", stringsAsFactors = FALSE)
test <- read.csv("test.csv", sep =",", stringsAsFactors = FALSE)

In [ ]:
boxplot(tmp$Fare1 ~ tmp$Age)

In [ ]:
# Split the name and create Salutation.

train$Salutation <- ifelse(grepl('Mr.',train$Name), 'Mr', 
                           ifelse(grepl('Mrs.', train$Name), 'Mrs', 
                                  ifelse(grepl('Miss.', train$Name), 'Miss', ifelse(grepl('Master.', train$Name), 'Master', 'Others'))))

temp1 <- train[,c(3,5,6,13)]
temp1 <- temp1 %>% 
  group_by(Pclass, Sex, Salutation) %>%
  summarise(medAge = round(median(Age, na.rm = TRUE),0))

train <- train %>%
left_join(temp1, by = c('Pclass', 'Sex' ,'Salutation')) 

train$Age <- ifelse(is.na(train$Age), train$medAge, train$Age)

In [ ]:
train %>% filter(Fare ) %>% nrow()

In [ ]:
# Replace the outlier fare value with a median fare value

temp2<- train %>%
  filter(Pclass == 1) %>%
  select(Pclass, Fare) %>%
  group_by(Pclass) %>%
  mutate(medFare = median(Fare))

#train$Fare <- ifelse(train$Fare > 400 | train$Fare == 0, 84.15469, train$Fare)

In [ ]:
# Detect numeric or alpha-numeric values in Ticket column 
train$fctrTicket <- ifelse(str_detect(train$Ticket, '[a-zA-Z]') == 1, '1', '0')
train$fctrTicket <- as.factor(train$fctrTicket)

In [ ]:
# Convert character to factor
train$Gender <- ifelse(train$Sex == 'male', '1', '0')
train$Survived <- as.factor(train$Survived)
train$Pclass <- as.factor(train$Pclass)
train$Gender <- as.factor(train$Gender)

In [ ]:
# Assumption 1: 
train$sRate <- ifelse(train$Pclass == '1' & train$Gender == '0', '1', '0')
train$sRate <- as.factor(train$sRate)

# Assumption 2:
train$psrFamily <- ifelse(train$SibSp > 0 | train$Parch > 0, '1','0')
train$psrFamily <- as.factor(train$psrFamily)

# Assumption 3:

train$sRate1 <- '0'
train$sRate1[train$Embarked == 'C'] <- '0'
train$sRate1[train$Embarked == 'Q'] <- '0'
train$sRate1[train$Embarked == 'S'] <- '0.5'
train$sRate1 <- as.factor(train$sRate1)

# Assumption 4:

train$ageBin = 0
train$ageBin[train$Age <= 10] = 1
train$ageBin[train$Age > 10 & train$Age <= 20] = 2
train$ageBin[train$Age > 20 & train$Age <= 30] = 3
train$ageBin[train$Age > 30 & train$Age <= 40] = 4
train$ageBin[train$Age > 40 & train$Age <= 50] = 5
train$ageBin[train$Age > 50] = 6 

train$ageBin <- as.factor(train$ageBin)

# Assumption 5:
train$sRate2 <- 0
train$sRate2[train$fctrTicket == 0] <- 1

In [ ]:
head(train)

In [ ]:
## fit a model using Random Forest
rf.fit <- randomForest(Survived ~ Pclass + Age + SibSp + Parch + Fare + sRate + psrFamily + 
                         ageBin + sRate1 + fctrTicket, data = train, importance = TRUE, ntree = 100)

pred <- predict(rf.fit, train)
table(pred, train$Survived)

In [ ]:
train %>% 
group_by(Pclass, fctrTicket) %>%
tally()